In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
file = "../Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# Find total player count
len(set(df["SN"]))

576

In [3]:
# Purchasing analysis (total)
# Run basic calculations to obtain number of unique items, average price, etc.
Unique_Items = len(set(df["Item Name"]))
Average_Cost = df["Price"].mean()
Total_Purchases = len(df["Purchase ID"])
Total_Revenue = df["Price"].sum()
# Create a summary data frame to hold the results
data_items = {
    "Unique Items": [Unique_Items],
    "Average Cost": [Average_Cost],
    "Total Purchases": [Total_Purchases],
    "Total Revenue": [Total_Revenue]
}
summary_df = pd.DataFrame(data_items, columns = ["Unique Items", "Average Cost", "Total Purchases", "Total Revenue"])
# Optional: give the displayed data cleaner formatting
#ROUND AVERAGE COST AND CHANGE AVERAGE COST AND TOTAL REVENUE TO DOLLARS
# Display the summary data frame
summary_df

,Unique Items,Average Cost,Total Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [4]:
# Gender Demographics
# Drop duplicate values
unique_player_df = df.drop_duplicates(subset = ["SN"])

#Count of players
new_df = pd.DataFrame(unique_player_df["Gender"].value_counts())
renamed_df = new_df.rename(columns = {"Gender": "Total Count"})

# Percentage of players
new2_df = pd.DataFrame(unique_player_df["Gender"].value_counts(normalize = True))
renamed2_df = new2_df.rename(columns = {"Gender": "Percentage"})

# Combine values into dataframe
gender_summary_df = renamed_df.merge(renamed2_df, how = "outer", left_index = True, right_index = True)

#Format and display dataframe
gender_summary_df
# FORMAT AS PERCENTAGE

,Total Count,Percentage
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [5]:
# Purchasing Analysis (Gender)
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
grouped_gender_df = df.groupby(["Gender"])

purchase_count = grouped_gender_df["Purchase ID"].count()
average_price = grouped_gender_df["Price"].mean()
total_value = grouped_gender_df["Price"].sum()

#Filter by gender then groupby person to find average purchase total per person per gender
df_female = df.loc[df["Gender"] == "Female", :].copy()
average_female = df_female.groupby("SN")["Price"].sum().mean()

df_male = df.loc[df["Gender"] == "Male", :].copy()
average_male = df_male.groupby("SN")["Price"].sum().mean()

df_other = df.loc[df["Gender"] == "Other / Non-Disclosed", :].copy()
average_other = df_other.groupby("SN")["Price"].sum().mean()

# Create a summary data frame to hold the results
data2_items = [average_female, average_male, average_other]
gender_summary_df = pd.DataFrame({"Purchase Count": purchase_count, 
                                  "Average Price": average_price,
                                  "Total Value": total_value,
                                  "Average Price per Person": data2_items})
# Optional: give the displayed data cleaner formatting
#$$$$$$$$$$
# Display the summary data frame
gender_summary_df

,Purchase Count,Average Price,Total Value,Average Price per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [6]:
# Age Demographics
# Establish bins for ages
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200]
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Categorize the existing players using the age bins. Hint: use pd.cut()
df["Age Bracket"] = pd.cut(df["Age"], bins, labels = age_groups, include_lowest = True)

# Calculate the numbers and percentages by age group
unique_player_df = df.drop_duplicates(subset = ["SN"])

new_df = pd.DataFrame(unique_player_df["Age Bracket"].value_counts())
renamed_df = new_df.rename(columns = {"Age Bracket": "Player Count"})

new2_df = pd.DataFrame(unique_player_df["Age Bracket"].value_counts(normalize = True))
renamed2_df = new2_df.rename(columns = {"Age Bracket": "Percentage"})

# Create a summary data frame to hold the results
age_summary_df = renamed_df.merge(renamed2_df, how = "outer", left_index = True, right_index = True)

# Format
age_summary_df = age_summary_df.reindex(index = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])

# Display Age Demographics Table
age_summary_df

,Player Count,Percentage
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


In [7]:
# Purchasing Analysis (Age)
# Bin the purchase_data data frame by age
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200]
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
df["Age Bracket"] = pd.cut(df["Age"], bins, labels = age_groups, include_lowest = True)

# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
grouped_age_df = df.groupby(["Age Bracket"])

purchase_count = grouped_age_df["Purchase ID"].count()
average_price = grouped_age_df["Price"].mean()
total_value = grouped_age_df["Price"].sum()

price_sum = grouped_age_df["Price"].sum()
player_count = age_summary_df["Player Count"]
avg_per_person = price_sum/player_count

# Create a summary data frame to hold the results
age_analysis_df = pd.DataFrame({"Purchase Count": purchase_count, 
                                  "Average Price": average_price,
                                  "Total Value": total_value,
                                  "Average Price per Person": avg_per_person})
# Optional: give the displayed data cleaner formatting

# Display the summary data frame
age_analysis_df

,Purchase Count,Average Price,Total Value,Average Price per Person
Age Bracket,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [8]:
# Top Spenders
# Run basic calculations to obtain the results in the table below
grouped_sn_df = df.groupby(["SN"])
total_price = grouped_sn_df["Price"].sum()
purchase_count = grouped_sn_df["Price"].size()

unique_player_df = df.drop_duplicates(subset = ["SN"]).copy()
unique_player_df = unique_player_df.set_index("SN")
unique_player_df["Total Price"] = total_price
unique_player_df["Purchase Count"] = purchase_count
top_spenders_df = unique_player_df.sort_values(by = "Total Price", ascending = False). head(5)
top_spenders_df

# Create a summary data frame to hold the results

# Sort the total purchase value column in descending order

# Optional: give the displayed data cleaner formatting

# Display a preview of the summary data frame

,Purchase ID,Age,Gender,Item ID,Item Name,Price,Age Bracket,Total Price,Purchase Count
SN,,,,,,,,,
Lisosia93,74,25,Male,89,"Blazefury, Protector of Delusions",4.64,25-29,18.96,5
Idastidru52,290,24,Male,147,"Hellreaver, Heirloom of Inception",4.93,20-24,15.45,4
Chamjask73,222,22,Female,178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,20-24,13.83,3
Iral74,128,21,Male,58,"Freak's Bite, Favor of Holy Might",4.14,20-24,13.62,4
Iskadarya95,148,20,Male,148,"Warmonger, Gift of Suffering's End",4.03,20-24,13.10,3


In [19]:
# Most Popular Items
# Retrieve the Item ID, Item Name, and Item Price columns
item_df = df[["Item ID", "Item Name", "Price"]]

# Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
item_gb = item_df.groupby(["Item ID", "Item Name"])

purchase_count = item_gb.count()
total_purchase_value = item_gb.sum()

# Create a summary data frame to hold the results
#Couldn't create new dataframe because items did not have index so used merge instead
item_working_df = pd.merge(item_df, total_purchase_value, on = ["Item ID", "Item Name"])
item_summary_df = pd.merge(item_working_df, purchase_count, on = ["Item ID", "Item Name"])
item_summary_df.columns = ["Item ID", "Item Name", "Price", "Total Purchase Value", "Purchase Count"]

#Manually getting format given by groupby in dataframe
item_summary_df = item_summary_df.drop_duplicates(subset = ["Item ID"])
item_summary_df = item_summary_df.set_index(["Item ID", "Item Name"])

# Sort the purchase count column in descending order
item_summary_df = item_summary_df.sort_values(by = "Purchase Count", ascending = False)
# Optional: give the displayed data cleaner formatting


# Display a preview of the summary data frame
item_summary_df.head()

,,Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
92,Final Critic,4.88,59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,50.76,12
108,"Extraction, Quickblade Of Trembling Hands",3.53,31.77,9
82,Nirvana,4.90,44.10,9
145,Fiery Glass Crusader,4.58,41.22,9


In [20]:
# Most Profitable Items
# Sort the above table by total purchase value in descending order
item_summary_df = item_summary_df.sort_values(by = "Total Purchase Value", ascending = False)

# Optional: give the displayed data cleaner formatting

#Display a preview of the data frame
item_summary_df.head()

,,Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
92,Final Critic,4.88,59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,50.76,12
82,Nirvana,4.90,44.10,9
145,Fiery Glass Crusader,4.58,41.22,9
103,Singed Scalpel,4.35,34.80,8
